<img src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/solutions-microsoft-logo-small.png?raw=true" alt="Microsoft">
<br>

# **SQL Server 2019 big data cluster Tutorial**
## **04 - Data Virtualization**

In this tutorial you will learn how to create and query Virtualized Data in a SQL Server big data cluster.  
- You'll start with creating a text file format, since that's the type of data you are reading in. 
- Next, you'll create a data source for the SQL Storage Pool, since that allows you to access the HDFS system in BDC. 
- Finally, you'll create an External Table, which uses the previous steps to access the data.


In [3]:
SELECT * FROM sys.external_tables

(0 rows affected)

Total execution time: 00:00:00.189

name,object_id,principal_id,schema_id,parent_object_id,type,type_desc,create_date,modify_date,is_ms_shipped,is_published,is_schema_published,max_column_id_used,uses_ansi_nulls,data_source_id,file_format_id,location,reject_type,reject_value,reject_sample_value,distribution_type,distribution_desc,sharding_col_id,remote_schema_name,remote_object_name,rejected_row_location


In [4]:
/* Clean up only - run this cell only if you are repeating the tutorial! */
USE WideWorldImporters;
GO

IF EXISTS(SELECT * FROM sys.external_data_sources WHERE name = 'SqlStoragePool')
BEGIN
    DROP EXTERNAL DATA SOURCE SqlStoragePool
END

IF EXISTS(SELECT * FROM sys.external_tables WHERE name = 'partner_customers_hdfs')
BEGIN
    DROP EXTERNAL TABLE partner_customers_hdfs
END

IF EXISTS(SELECT * FROM sys.external_file_formats WHERE name = 'csv_file')
BEGIN
    DROP EXTERNAL FILE FORMAT csv_file
END


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.227

In [5]:
/* Create External File Format */

USE WideWorldImporters;
GO

CREATE EXTERNAL FILE FORMAT csv_file
WITH (
    FORMAT_TYPE = DELIMITEDTEXT,
    FORMAT_OPTIONS(
        FIELD_TERMINATOR = ',',
        STRING_DELIMITER = '0x22',
        FIRST_ROW = 2,
        USE_TYPE_DEFAULT = TRUE)
);
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.459

In [6]:
/* Create External Data Source to the Storage Pool */
CREATE EXTERNAL DATA SOURCE SqlStoragePool
WITH (LOCATION = 'sqlhdfs://controller-svc/default');

Commands completed successfully.

Total execution time: 00:00:00.171

In [7]:
/* Create an External Table that can read from the Storage Pool File Location */
CREATE EXTERNAL TABLE [partner_customers_hdfs]
    ("CustomerSource" VARCHAR(250) 
    , "CustomerName" VARCHAR(250) 
    , "EmailAddress" VARCHAR(250))
    WITH
    (
        DATA_SOURCE = SqlStoragePool,
        LOCATION = '/partner_customers',
        FILE_FORMAT = csv_file
    );
GO

Commands completed successfully.

Total execution time: 00:00:00.384

In [8]:
/* Read Data from HDFS using only T-SQL */

SELECT TOP 10 CustomerSource
, CustomerName
, EMailAddress
    FROM [partner_customers_hdfs] hdfs
WHERE EmailAddress LIKE '%wingtip%'
ORDER BY CustomerSource, CustomerName;
GO


(10 rows affected)

Total execution time: 00:00:02.762

CustomerSource,CustomerName,EMailAddress
AdventureWorks,Ã…Å¡ani Nair,Ã¥Å¡ani@wingtiptoys.com
AdventureWorks,Ã…Å¡ani Sen,Ã¥Å¡ani@wingtiptoys.com
AdventureWorks,Aakriti Bhamidipati,aakriti@wingtiptoys.com
AdventureWorks,Aamdaal Kamasamudram,aamdaal@wingtiptoys.com
AdventureWorks,Abel Pirvu,abel@wingtiptoys.com
AdventureWorks,Abhaya Rambhatla,abhaya@wingtiptoys.com
AdventureWorks,Abhra Thakur,abhra@wingtiptoys.com
AdventureWorks,Adam Balaz,adam@wingtiptoys.com
AdventureWorks,Adirake Narkbunnum,adirake@wingtiptoys.com
AdventureWorks,Adirake Saenamuang,adirake@wingtiptoys.com


In [9]:
/* Now Join Those to show customers we currently have in a SQL Server Database 
and the Category they qre in the External Table */
USE WideWorldImporters;
GO

SELECT TOP 10 a.FullName
  , b.CustomerSource
  FROM Application.People a
  INNER JOIN partner_customers_hdfs b  ON a.FullName = b.CustomerName
  ORDER BY FullName ASC;
  GO

Commands completed successfully.

(10 rows affected)

Total execution time: 00:00:01.807

FullName,CustomerSource
Aahlada Thota,AdventureWorks
Aakarsha Nookala,AdventureWorks
Aakriti Bhamidipati,AdventureWorks
Aamdaal Kamasamudram,AdventureWorks
Abel Pirvu,AdventureWorks
Abhaya Rambhatla,AdventureWorks
Abhra Thakur,AdventureWorks
Adam Balaz,AdventureWorks
Adam Dvorak,AdventureWorks
Adam Kubat,AdventureWorks


## **Next Steps: Continue on to Working with the SQL Server Data Pool**

Now you're ready to open the next Python Notebook - `bdc_tutorial_05.ipynb` - to learn how to create and work with a Data Mart.